# HomeWork1
#### Panayotis Papoutsis

## Librairies 

In [138]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle
import numpy as np

from sklearn.metrics import mean_squared_error

### Q1. Downloading the data

Read the data for January. How many records are there?

In [139]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
print("There are",len(df), "records")




There are 1154112 records


### Q2. Computing duration
What's the average trip duration in January?

In [140]:
df.dtypes


dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [141]:
df.columns


Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [142]:
df['duration'] = df.dropOff_datetime -df.pickup_datetime
df.duration = df.duration.apply(lambda td : td.total_seconds()/60)
print("The average trip duration is", np.around(df.duration.mean(),2),"minutes.")


The average trip duration is 19.17 minutes.


## Data preparation

In [143]:
print("We dropped",len(df[~((df.duration >= 1) & (df.duration <= 60))]) ,"records")

We dropped 44286 records


In [144]:
df = df[((df.duration >= 1) & (df.duration <= 60))]

### Q3. Missing values
What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [145]:
print("The percentage of missing values for the pickup location ID is ", np.around((df.PUlocationID.isnull().sum()/len(df))*100,2),"%.")


The percentage of missing values for the pickup location ID is  83.53 %.


In [146]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna(-1).astype('int')

### Q4. One-hot encoding
What's the dimensionality of this matrix? (The number of columns).

In [147]:
categorical = ['PUlocationID','DOlocationID']
df[categorical] = df[categorical].astype('str')
dv = DictVectorizer()

train_dicts = df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts) 
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [148]:
print("The dimensionality of the design matrix is", len(dv.feature_names_),".")

The dimensionality of the design matrix is 525 .


### Q5. Training a model
What's the RMSE on train?

In [149]:
target = 'duration'

y_train = df[target].values

In [150]:
lr = LinearRegression()

lr.fit(X_train,y_train)
y_pred = lr.predict(X_train)

print("The RMSE on train is", np.round(mean_squared_error(y_train,y_pred,squared=False),2))

The RMSE on train is 10.53


### Q6. Evaluating the model
What's the RMSE on validation?

In [151]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [152]:
df_val = read_data('./data/fhv_tripdata_2021-02.parquet')

In [153]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts) 

In [154]:

y_val = df_val.duration.values
y_pred = lr.predict(X_val)
print("The RMSE is", np.round(mean_squared_error(y_val,y_pred,squared=False),2))

The RMSE is 11.01
